# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2927e19,"""16326606554602…","""../../../../da…","""69808131098037…","""16148878945884…","""13747558808755…",false
1.4856e19,"""10894232297879…","""../../../../da…","""54167904556947…","""16148878945884…","""81020533646508…",true
5.6458e17,"""65535505530279…","""../../../../da…","""10380689543412…","""14685618251085…","""13747558808755…",false
4.0723e18,"""75048678520548…","""../../../../da…","""54167904556947…","""10893807971399…","""13747558808755…",false
1.4677e19,"""10815269458870…","""../../../../da…","""69808131098037…","""10893807971399…","""11848394332720…",true
1.6330e19,"""13213212117438…","""../../../../da…","""10380689543412…","""13556167894771…",null,false
8.7675e18,"""17930162555958…","""../../../../da…","""10380689543412…","""10893807971399…","""11848394332720…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""12674127417790…","""../../../../da…","""10525479500256…","""FirstMortgage3…","""60148436066776…","""Rejected"""
1.4856e19,"""47281164881054…","""../../../../da…","""18174008791240…","""FirstMortgage3…","""82013432002092…","""Accepted"""
5.6458e17,"""11164993895411…","""../../../../da…","""13912078629636…","""MoneyMarketSav…","""60148436066776…","""Rejected"""
4.0723e18,"""10536464129096…","""../../../../da…","""18174008791240…","""BasicChecking""","""60148436066776…","""Rejected"""
1.4677e19,"""30166919412541…","""../../../../da…","""10525479500256…","""BasicChecking""","""43517659467149…","""Accepted"""
1.6330e19,"""17811250415843…","""../../../../da…","""13912078629636…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""80566578146808…","""../../../../da…","""13912078629636…","""BasicChecking""","""43517659467149…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""51022053017991…","""../../../../da…","""22908421515463…","""FirstMortgage3…","""11070735192924…","""Rejected"""
1.4856e19,"""22509607729529…","""../../../../da…","""14913285745009…","""FirstMortgage3…","""87953045676512…","""Accepted"""
5.6458e17,"""12741394156958…","""../../../../da…","""12791549990079…","""MoneyMarketSav…","""11070735192924…","""Rejected"""
4.0723e18,"""19999555687832…","""../../../../da…","""14913285745009…","""BasicChecking""","""11070735192924…","""Rejected"""
1.4677e19,"""99378810782500…","""../../../../da…","""22908421515463…","""BasicChecking""","""17413216099457…","""Accepted"""
1.6330e19,"""18352496983341…","""../../../../da…","""12791549990079…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""16104359481884…","""../../../../da…","""12791549990079…","""BasicChecking""","""17413216099457…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""15500537363418…","""61553610260354…","""FirstMortgage3…","""14608685220174…","""Rejected"""
1.4856e19,"""16760113396886…","""18275864373165…","""FirstMortgage3…","""28880109595722…","""Accepted"""
5.6458e17,"""83743029975003…","""34085234642677…","""MoneyMarketSav…","""14608685220174…","""Rejected"""
4.0723e18,"""94639722361197…","""18275864373165…","""BasicChecking""","""14608685220174…","""Rejected"""
1.4677e19,"""18041475748479…","""61553610260354…","""BasicChecking""","""17516838261600…","""Accepted"""
1.6330e19,"""20877775560059…","""34085234642677…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""17211437483383…","""34085234642677…","""BasicChecking""","""17516838261600…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'